In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.precision', 4)

print("✓ Librerie caricate con successo")

# Load data
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Data')

print("=" * 70)
print("SRK/T2 IOL FORMULA IMPLEMENTATION")
print("=" * 70)
print(f"Number of patients: {len(df)}")
print()

# Calculate average K
df['K_avg'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2

✓ Librerie caricate con successo
SRK/T2 IOL FORMULA IMPLEMENTATION
Number of patients: 96



In [12]:
def calculate_SRKT2(AL, K_avg, IOL_power, A_constant, nc=1.333, k_index=1.3375):
    """
    SRK/T2 Formula (Sheard et al. 2010)
    Modified version of SRK/T formula
    
    Parameters:
    -----------
    AL : float - Axial length (mm)
    K_avg : float - Average keratometry (D)
    IOL_power : float - IOL power (D)
    A_constant : float - A-constant for the IOL
    nc : float - Corneal refractive index (default 1.333)
    k_index : float - Keratometric index (default 1.3375)
    
    Returns:
    --------
    float - Predicted postoperative refraction (D)
    """
    # Constants
    na = 1.336  # Aqueous/vitreous refractive index
    V = 12      # Vertex distance (mm)
    ncm1 = nc - 1
    
    # Calculate corneal radius from keratometry
    r = (k_index - 1) * 1000 / K_avg
    
    # Axial length correction for long eyes
    if AL <= 24.2:
        LCOR = AL
    else:
        LCOR = 3.446 + 1.716 * AL - 0.0237 * AL * AL
    
    # H2 calculation (corneal height) - Sheard's modification
    H2 = -10.326 + 0.32630 * LCOR + 0.13533 * K_avg
    
    # ACD (Anterior Chamber Depth) estimation
    ACD_const = 0.62467 * A_constant - 68.747
    offset = ACD_const - 3.336
    ACD_est = H2 + offset
    
    # Retinal thickness correction
    RETHICK = 0.65696 - 0.02029 * AL
    LOPT = AL + RETHICK  # Optical axial length
    
    # SRK/T2 refraction calculation
    numerator = (1000 * na * (na * r - ncm1 * LOPT) - 
                 IOL_power * (LOPT - ACD_est) * (na * r - ncm1 * ACD_est))
    
    denominator = (na * (V * (na * r - ncm1 * LOPT) + LOPT * r) - 
                   0.001 * IOL_power * (LOPT - ACD_est) * 
                   (V * (na * r - ncm1 * ACD_est) + ACD_est * r))
    
    return numerator / denominator

print("✓ Formula SRK/T2 definita")

✓ Formula SRK/T2 definita


In [13]:
print("CALCULATING SRK/T2 PREDICTIONS...")
print("-" * 70)

# Calculate predictions for all patients
df['SRKT2_Prediction'] = df.apply(
    lambda row: calculate_SRKT2(
        AL=row['Bio-AL'],
        K_avg=row['K_avg'],
        IOL_power=row['IOL Power'],
        A_constant=row['A-Constant']
    ), axis=1
)

# Calculate prediction errors
df['Prediction_Error'] = df['PostOP Spherical Equivalent'] - df['SRKT2_Prediction']
df['Absolute_Error'] = abs(df['Prediction_Error'])

print(f"✓ Predizioni calcolate per {len(df)} pazienti")

# Calculate metrics
mae = df['Absolute_Error'].mean()
me = df['Prediction_Error'].mean()
std = df['Prediction_Error'].std()
median_ae = df['Absolute_Error'].median()

print("\n📊 SRK/T2 FORMULA PERFORMANCE METRICS:")
print("=" * 70)
print(f"  Mean Absolute Error (MAE):     {mae:.4f} D")
print(f"  Mean Error (ME):                {me:+.4f} D")
print(f"  Standard Deviation (SD):        {std:.4f} D")
print(f"  Median Absolute Error:          {median_ae:.4f} D")

# Calculate clinical accuracy
within_025 = (df['Absolute_Error'] <= 0.25).sum() / len(df) * 100
within_050 = (df['Absolute_Error'] <= 0.50).sum() / len(df) * 100
within_075 = (df['Absolute_Error'] <= 0.75).sum() / len(df) * 100
within_100 = (df['Absolute_Error'] <= 1.00).sum() / len(df) * 100

print("\n📈 CLINICAL ACCURACY:")
print("-" * 70)
print(f"  Within ±0.25 D:  {within_025:.1f}% of eyes")
print(f"  Within ±0.50 D:  {within_050:.1f}% of eyes")
print(f"  Within ±0.75 D:  {within_075:.1f}% of eyes")
print(f"  Within ±1.00 D:  {within_100:.1f}% of eyes")

CALCULATING SRK/T2 PREDICTIONS...
----------------------------------------------------------------------
✓ Predizioni calcolate per 96 pazienti

📊 SRK/T2 FORMULA PERFORMANCE METRICS:
  Mean Absolute Error (MAE):     1.3591 D
  Mean Error (ME):                -0.2915 D
  Standard Deviation (SD):        1.7471 D
  Median Absolute Error:          1.0311 D

📈 CLINICAL ACCURACY:
----------------------------------------------------------------------
  Within ±0.25 D:  13.5% of eyes
  Within ±0.50 D:  26.0% of eyes
  Within ±0.75 D:  35.4% of eyes
  Within ±1.00 D:  49.0% of eyes
